## Ch1

https://www.kaggle.com/c/house-prices-advanced-regression-techniques

House Prices - Advanced Regression Techniques

In [22]:
import pandas as pd

train_data = pd.read_csv('./data/ch1/train.csv')
test_data = pd.read_csv('./data/ch1/test.csv')
train_data[:5]
# 81列，看起来要素很多

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [23]:
print(train_data.shape)
test_data.shape
# 训练集1460行
# 测试集1459行

(1460, 81)


(1459, 80)

In [24]:
train_data.iloc[0:4, [0, 1, 2, 3, -3, -2, -1]]  # 四行数据  # 后面是列
# 第一列 Id 对我们的预测无帮助，不要 它不携带任何用于预测的信息

,Id,MSSubClass,MSZoning,LotFrontage,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,WD,Normal,208500
1,2,20,RL,80.0,WD,Normal,181500
2,3,60,RL,68.0,WD,Normal,223500
3,4,70,RL,60.0,WD,Abnorml,140000


In [25]:
# 去掉第一列Id
# 去掉训练集的标签列 SalePrice 房价
all_features = pd.concat((train_data.iloc[:, 1:-1], test_data.iloc[:, 1:]))
all_features.shape

(2919, 79)

开始训练前,一个是需要对应的模型这是不用说的
其次就是数据要进行处理
比如说，有无空值NAN？是去掉还是填充？填充平均还是中位？
有无 离散值？ 比如说男女性别 male female  ---- 独热编码
有无不合理的值？比如相较于其他的值差异很大等

In [37]:
"""
将所有缺失的值替换为相应特征的平均值。
然后，为了将所有特征放在一个共同的尺度上,将特征重新缩放到零均值和单位方差来  标准化数据
"""
# 若无法获得测试数据，则可根据训练数据计算均值和标准差
numeric_features = all_features.dtypes[all_features.dtypes != 'object'].index
all_features[numeric_features] = all_features[numeric_features].apply(
    lambda x: (x - x.mean()) / (x.std()))
# 在标准化数据之后，所有均值消失，因此我们可以将缺失值设置为0
all_features[numeric_features] = all_features[numeric_features].fillna(0)

In [36]:
# “Dummy_na=True”将“na”（缺失值）视为有效的特征值，并为其创建指示符特征
all_features = pd.get_dummies(all_features, dummy_na=True)
print(all_features.dtypes)
all_features.shape

MSSubClass               float64
LotFrontage              float64
LotArea                  float64
OverallQual              float64
OverallCond              float64
                          ...   
SaleCondition_Alloca     float64
SaleCondition_Family     float64
SaleCondition_Normal     float64
SaleCondition_Partial    float64
SaleCondition_nan        float64
Length: 330, dtype: object


(2919, 330)

In [34]:
import torch

# 特征的总数量从79个增加到330个,转换为张量表示用于训练
n_train = train_data.shape[0]
train_features = torch.tensor(all_features[:n_train].values, dtype=torch.float32)
test_features = torch.tensor(all_features[n_train:].values, dtype=torch.float32)
train_labels = torch.tensor(train_data.SalePrice.values.reshape(-1, 1), dtype=torch.float32)
train_labels[:3]

tensor([[208500.],
        [181500.],
        [223500.]])

In [38]:
from torch import nn

# 训练
# 1. 损失平方的线性模型
loss = nn.MSELoss()
in_features = train_features.shape[1]
print(in_features)


def get_net():
    net = nn.Sequential(nn.Linear(in_features, 1))  # 线性，全连接层
    return net


get_net()

330


Sequential(
  (0): Linear(in_features=330, out_features=1, bias=True)
)

In [39]:
def log_rmse(net, features, labels):
    # 为了在取对数时进一步稳定该值，将小于1的值设置为1
    clipped_preds = torch.clamp(net(features), 1, float('inf'))
    rmse = torch.sqrt(loss(torch.log(clipped_preds),
                           torch.log(labels)))
    return rmse.item()


TypeError: log_rmse() missing 3 required positional arguments: 'net', 'features', and 'labels'

In [ ]:
from torch.utils.data import DataLoader, TensorDataset


# Adam优化器的主要吸引力在于它对初始学习率不那么敏感。
def train(net, train_features, train_labels, test_features, test_labels,
          num_epochs, learning_rate, weight_decay, batch_size):
    train_ls, test_ls = [], []
    # train_iter = d2l.load_array((train_features, train_labels), batch_size)
    # 将数据转换为 TensorDataset
    train_dataset = TensorDataset(train_features, train_labels)
    train_iter = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    # 这里使用的是Adam优化算法
    optimizer = torch.optim.Adam(net.parameters(),
                                 lr=learning_rate,
                                 weight_decay=weight_decay)
    for epoch in range(num_epochs):
        for X, y in train_iter:
            optimizer.zero_grad()
            l = loss(net(X), y)
            l.backward()
            optimizer.step()
        train_ls.append(log_rmse(net, train_features, train_labels))
        if test_labels is not None:
            test_ls.append(log_rmse(net, test_features, test_labels))
    return train_ls, test_ls